In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset,DataLoader, Subset


from torch.optim.lr_scheduler import ReduceLROnPlateau

from sklearn.model_selection import train_test_split, KFold
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter

# Добавьте это в начале кода (после импортов)



In [ ]:
train_data,test_data = pd.read_csv("/content/drive/MyDrive/Datasets and study/Дата для домашки 5 по CV/train.csv"), pd.read_csv("/content/drive/MyDrive/Datasets and study/Дата для домашки 5 по CV/test.csv")


In [ ]:
labels = train_data['target']
features = train_data.drop(columns = ["target"])
X_train, X_test, y_train, y_test = train_test_split(features,labels, train_size=0.90, random_state=42, shuffle=True, stratify=labels)
input_size = features.shape[1]

In [ ]:
test_dataset = TensorDataset(
    torch.FloatTensor(X_test.values),
    torch.LongTensor(y_test.values)
)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

train_dataset = TensorDataset(
    torch.FloatTensor(X_train.values),
    torch.LongTensor(y_train.values)
)


In [ ]:
tb_writer = SummaryWriter()

In [ ]:
def train_one_epoch(model, training_loader, optimizer, F_loss):
    running_loss = 0.
    last_loss = 0.
    for i, data in tqdm(enumerate(training_loader)):
        inputs, labels = data
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = F_loss(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    last_loss = running_loss / len(training_loader)

    return last_loss



In [ ]:
model = nn.Sequential(
          nn.Linear(input_size, 2048),
          nn.BatchNorm1d(2048),
          nn.Dropout(0.6),
          nn.LeakyReLU(0.1),

          nn.Linear(2048, 1024),
          nn.BatchNorm1d(1024),
          nn.Dropout(0.4),
          nn.LeakyReLU(0.1),

          nn.Linear(1024, 512),
          nn.BatchNorm1d(512),
          nn.Dropout(0.2),
          nn.LeakyReLU(0.1),

          nn.Linear(512, 2)
      )
optimizer = torch.optim.Adam(model.parameters(),weight_decay=1e-5)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(DEVICE)

Sequential(
  (0): Linear(in_features=16, out_features=2048, bias=True)
  (1): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): Dropout(p=0.6, inplace=False)
  (3): LeakyReLU(negative_slope=0.1)
  (4): Linear(in_features=2048, out_features=1024, bias=True)
  (5): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (6): Dropout(p=0.4, inplace=False)
  (7): LeakyReLU(negative_slope=0.1)
  (8): Linear(in_features=1024, out_features=512, bias=True)
  (9): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (10): Dropout(p=0.2, inplace=False)
  (11): LeakyReLU(negative_slope=0.1)
  (12): Linear(in_features=512, out_features=2, bias=True)
)

In [ ]:
splits = 10
EPOCHS = 20
kfold = KFold(n_splits=splits, shuffle=True)
F_loss = torch.nn.CrossEntropyLoss()
#scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.5)
#models = []

for fold, (train_ids, test_ids) in enumerate(kfold.split(train_dataset)):
    print(f'Fold {fold + 1}')



    # Create subsets
    train_subset = torch.utils.data.Subset(train_dataset, train_ids)
    test_subset = torch.utils.data.Subset(train_dataset, test_ids)

    trainset = DataLoader(train_subset, batch_size=32, shuffle=True)
    testset = DataLoader(test_subset, batch_size=32, shuffle=True)

    for epoch in range(EPOCHS):
        print('EPOCH {}:'.format(epoch + 1))
        model.train(True)

        avg_loss = train_one_epoch(model, trainset, optimizer, F_loss)
        running_vloss = avg_loss

    model.eval()
    with torch.no_grad():
        for i, vdata in enumerate(testset):
            vinputs, vlabels = vdata
            vinputs = vinputs.to(DEVICE)
            vlabels = vlabels.to(DEVICE)
            voutputs = model(vinputs)
            vloss = F_loss(voutputs, vlabels)
            running_vloss += vloss.item()


            # scheduler.step(vloss)  # Обновляем LR
            # if optimizer.param_groups[0]['lr'] < 1e-6:
            #     print("LR слишком мал, останавливаем обучение!")
            #     break
        avg_vloss = running_vloss / (i + 1)
        print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))

    #models.append(model)

Fold 1
EPOCH 1:


33it [00:00, 263.59it/s]


EPOCH 2:


33it [00:00, 270.16it/s]


EPOCH 3:


...

EPOCH 19:


33it [00:00, 254.50it/s]


EPOCH 20:


33it [00:00, 260.29it/s]

LOSS train 0.3522406658439925 valid 0.6100435217447353


In [ ]:
tb_writer.close()

In [ ]:
def evaluate_model(model):
    model.eval()
    # 2. Инициализируем метрики
    running_vloss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for vinputs, vlabels in test_loader:  # 3. Используем DataLoader
            vinputs = vinputs.to(DEVICE)
            vlabels = vlabels.to(DEVICE)
            voutputs = model(vinputs)

            # 4. Вычисляем лосс
            vloss = F_loss(voutputs, vlabels)
            running_vloss += vloss.item() * vinputs.size(0)  # Умножаем на размер батча

            # 5. Вычисляем accuracy
            _, predicted = torch.max(voutputs.data, 1)
            total += vlabels.size(0)
            correct += (predicted == vlabels).sum().item()

    # 6. Вычисляем средние значения
    avg_loss = running_vloss / total
    accuracy = correct / total

    print(f' Test Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}')

In [ ]:
evaluate_model(model)

 Test Loss: 0.4878, Accuracy: 0.7674


In [ ]:
def evaluate_all_k_fold_models(models):
  for model_number,(my_model,avr_loss) in enumerate(models):

    my_model.eval()


    # 2. Инициализируем метрики
    running_vloss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for vinputs, vlabels in test_loader:  # 3. Используем DataLoader
            voutputs = model(vinputs)

            # 4. Вычисляем лосс
            vloss = F_loss(voutputs, vlabels)
            running_vloss += vloss.item() * vinputs.size(0)  # Умножаем на размер батча

            # 5. Вычисляем accuracy
            _, predicted = torch.max(voutputs.data, 1)
            total += vlabels.size(0)
            correct += (predicted == vlabels).sum().item()

    # 6. Вычисляем средние значения
    avg_loss = running_vloss / total
    accuracy = correct / total

    print(f'Model number: {model_number}, Test Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}')

In [ ]:
def ensemble_predict(models, X):
    avg_logits = torch.zeros((X.shape[0], 2))  # Для 2 классов
    for model, _ in models:
        model.eval()
        with torch.no_grad():
            logits = model(X)
            avg_logits += logits

    avg_logits /= len(models)
    return torch.max(avg_logits, dim=1)

def evaluate_model_ensemble(models):
  running_vloss = 0
  total = 0
  correct = 0
  for vinputs, vlabels in test_loader:  # 3. Используем DataLoader
      voutputs =  ensemble_predict(models,vinputs)
      # 4. Вычисляем лосс
      vloss = F_loss(voutputs, vlabels)
      running_vloss += vloss.item() * vinputs.size(0)  # Умножаем на размер батча

      # 5. Вычисляем accuracy
  _, predicted = torch.max(voutputs.data, 1)
  total += vlabels.size(0)
  correct += (predicted == vlabels).sum().item()

# 6. Вычисляем средние значения
  avg_loss = running_vloss / total
  accuracy = correct / total

  print(f' Test Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}')

In [ ]:
def ensemble_predict_2(models, X):
    avg_logits = torch.zeros((X.shape[0], 2), device=X.device)  # Для 2 классов
    for model, _ in models:
        model.eval()
        with torch.no_grad():
            logits = model(X)
            avg_logits += logits

    avg_logits /= len(models)
    return avg_logits  # Возвращаем логиты, а не индексы классов

def evaluate_model_ensemble_2(models):
    running_vloss = 0.0
    total_samples = 0

    for vinputs, vlabels in test_loader:
        # Перенос данных на тот же device, что и модель
        vinputs = vinputs.to(next(models[0][0].parameters()).device)
        vlabels = vlabels.to(next(models[0][0].parameters()).device)

        # Получаем логиты от ансамбля
        voutputs = ensemble_predict_2(models, vinputs)

        # Вычисляем лосс (теперь передаем логиты)
        vloss = F_loss(voutputs, vlabels)
        running_vloss += vloss.item() * vinputs.size(0)
        total_samples += vinputs.size(0)

    # Возвращаем средний лосс
    return running_vloss / total_samples

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=runs

In [ ]:
evaluate_model_ensemble(models)

 Test Loss: 40.5487, Accuracy: 1.0000


In [ ]:
evaluate_model_ensemble_2(models)

0.6238256436127882

In [ ]:
def ensemble_predict(models, X):
    # Проверяем и исправляем размерность
    if X.dim() == 1:
        X = X.unsqueeze(0)  # Добавляем batch-размер если нет
    elif X.dim() == 2 and X.shape[0] == 1:
        pass  # Уже правильный формат [1, features]
    else:
        X = X.reshape(-1, X.shape[-1])  # Решаем проблему с BatchNorm

    avg_logits = torch.zeros((X.shape[0], 2), device=X.device)

    for model, _ in models:
        model.eval()
        with torch.no_grad():
            logits = model(X)
            avg_logits += logits

    avg_logits /= len(models)
    return avg_logits  # Возвращаем логиты, а не результат max

def predict_models_ensemble(models, dataloader):
    predictions = []
    for batch in dataloader:
        inputs = batch[0] if isinstance(batch, (tuple, list)) else batch

        # Проверка размерности
        if inputs.dim() == 1:
            inputs = inputs.unsqueeze(0)

        outputs = ensemble_predict(models, inputs)
        _, preds = torch.max(outputs, 1)
        predictions.extend(preds.cpu().numpy())

    return predictions

def predict(models, dataloader):
    predictions = []
    model = models[0][0]
    for batch in dataloader:
        inputs = batch[0] if isinstance(batch, (tuple, list)) else batch

        # Проверка размерности
        if inputs.dim() == 1:
            inputs = inputs.unsqueeze(0)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        predictions.extend(preds.cpu().numpy())

    return predictions

In [ ]:
test_tensor = torch.FloatTensor(test_data.values)
test_dataset = TensorDataset(test_tensor)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False,drop_last=False)


430


In [ ]:
predictions = predict(models, test_loader)
# 7. Сохраняем в CSV
pd.DataFrame(predictions).to_csv('answers.csv', header=False, index=False, encoding='utf-8')
saved_data = pd.read_csv('/content/answers.csv', header=None, encoding='utf-8')
print(len(saved_data))

430
